In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
train=pd.read_csv('train.csv' )
test=pd.read_csv('test.csv')

In [3]:
train.head()
train.shape,test.shape

((31962, 3), (17197, 2))

In [4]:
def clean(text):
    text=text.lower()
    text=re.sub('[^a-zA-z#]',' ',text)
    text=re.sub('@"([^\w]*Admin[^\w]*)+|[|\\^&\r\n]+"',' ',text)
    text=re.sub("[''""...]",'',text)
    text=re.sub("\n",'',text)
    return text
combi=train.append(test,ignore_index=False)
combi.head()
combi.shape

(49159, 3)

In [5]:
combi['tweet']=combi['tweet'].apply(lambda x : clean(x))

In [6]:
combi['tweet']=combi['tweet'].str.replace("user"," ")
combi.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so s...
1,2,0.0,thanks for #lyft credit i can t use caus...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide society now #motivation


In [7]:
train=combi.head(31961)

In [16]:
train.head()
len(train.label)
train.tail()
len(train['tweet'][0])

99

In [9]:
test=combi[31962:]

In [10]:
test.dropna(axis=1,inplace=True)
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,#white #supremacists want everyone to see ...
2,31965,safe ways to heal your #acne #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,rd #bihday to my amazing hilarious #nephew...


In [173]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(stop_words='english')
x_train_=cv.fit_transform(train.tweet)
x_train_.shape,train.label.shape

((31961, 37356), (31961,))

In [174]:
from sklearn.model_selection import train_test_split
(x_train,x_test,y_train,y_test)=train_test_split(x_train_,train.label,test_size=0.2,random_state=0)

In [175]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

((25568, 37356), (25568,), (6393, 37356), (6393,))

In [128]:
from keras.models import Sequential
from keras.layers import Dense,Dropout

Using TensorFlow backend.


In [159]:
model=Sequential()
model.add(Dense(1024,activation='relu',input_dim=37356))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [160]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [178]:
history=model.fit(x_train,y_train,batch_size=128,epochs=5,validation_split=0.2)

Train on 20454 samples, validate on 5114 samples
Epoch 1/5
20454/20454 [==============================] - 123s 6ms/step - loss: 0.0535 - accuracy: 0.9938 - val_loss: 0.7836 - val_accuracy: 0.9415
Epoch 2/5
20454/20454 [==============================] - 122s 6ms/step - loss: 0.0556 - accuracy: 0.9924 - val_loss: 0.3439 - val_accuracy: 0.9183
Epoch 3/5
20454/20454 [==============================] - 114s 6ms/step - loss: 0.0493 - accuracy: 0.9940 - val_loss: 0.6056 - val_accuracy: 0.9302
Epoch 4/5
20454/20454 [==============================] - 108s 5ms/step - loss: 0.0463 - accuracy: 0.9936 - val_loss: 1.1743 - val_accuracy: 0.9241
Epoch 5/5
20454/20454 [==============================] - 105s 5ms/step - loss: 0.0424 - accuracy: 0.9937 - val_loss: 0.9926 - val_accuracy: 0.9163


In [176]:
from sklearn.metrics import f1_score
y_hat=model.predict_classes(x_test)
print(f1_score(y_hat,y_test))

0.611111111111111


In [ ]:
x_test=cv.transform(test.tweet)
prediction=model.predict_classes(x_test)

In [ ]:
test['label']=prediction
test.drop(test['tweet'],axis=1)

In [ ]:
test.t_csv("Submission.csv",index=False)

In [13]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

In [59]:
vocab_size=10000
max_doc_len=50
embedding_length=10
x_train=[one_hot(tweet,vocab_size) for tweet in train['tweet']]
y_train=train['label']

In [60]:
x_train=pad_sequences(x_train,truncating='post',padding='post',maxlen=max_doc_len)

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [28]:
(x_train,x_test,y_train,y_test)=train_test_split(x_train,y_train,test_size=0.1,random_state=18)

In [29]:
from keras.models import Sequential
from keras.layers import Dense,SimpleRNN,LSTM,GRU,Dropout,Embedding

In [61]:
model=Sequential()
model.add(Embedding(vocab_size,embedding_length,input_length=max_doc_len))
model.add(SimpleRNN(32,activation='relu',return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,activation='relu',return_sequences=True))
model.add(Dropout(0.4))
model.add(GRU(64,activation='relu',return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(32,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1,activation='sigmoid'))

In [62]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [63]:
model.fit(x_train,y_train,batch_size=128,epochs=10,validation_split=0.2)

Train on 25568 samples, validate on 6393 samples
Epoch 1/10
25568/25568 [==============================] - 32s 1ms/step - loss: 22.3633 - accuracy: 0.9217 - val_loss: 0.2437 - val_accuracy: 0.9305
Epoch 2/10
25568/25568 [==============================] - 31s 1ms/step - loss: 0.2870 - accuracy: 0.9296 - val_loss: 0.2232 - val_accuracy: 0.9305
Epoch 3/10
25568/25568 [==============================] - 33s 1ms/step - loss: 3458.0842 - accuracy: 0.9296 - val_loss: 6.4670 - val_accuracy: 0.9305
Epoch 4/10
25568/25568 [==============================] - 33s 1ms/step - loss: 20883.0578 - accuracy: 0.9239 - val_loss: 6864.5047 - val_accuracy: 0.9305
Epoch 5/10
25568/25568 [==============================] - 29s 1ms/step - loss: 3682.3906 - accuracy: 0.9105 - val_loss: 1601.8802 - val_accuracy: 0.9305
Epoch 6/10
25568/25568 [==============================] - 29s 1ms/step - loss: 12672.3845 - accuracy: 0.8977 - val_loss: 2421.0765 - val_accuracy: 0.9305
Epoch 7/10
25568/25568 [=====================

In [43]:
print(f1_score(model.predict_classes(x_test),y_test))

0.6180904522613067


In [44]:
x_test=[one_hot(tweet,vocab_size) for tweet in test['tweet']]
x_test=pad_sequences(x_test,truncating='post',padding='post',maxlen=max_doc_len)

In [64]:
test['label']=model.predict_classes(x_test)

In [66]:
#test.drop('tweet',axis=1,inplace=True)
test.head()

,id,label
0,31963,0
1,31964,0
2,31965,0
3,31966,0
4,31967,0


In [67]:
test.to_csv("Submission.csv",index=False)